In [4]:
from ultralytics import YOLO
from dataset import YOLO11dataset
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader
from sklearn.model_selection import train_test_split
import os
import shutil
import tqdm

In [4]:
def copy_files(file_names, src_dir, dst_dir):
    for fname in file_names:
        # Copy image
        shutil.copy2(
            os.path.join(src_dir, "images", fname),
            os.path.join(dst_dir, "images", fname)
        )
        # Copy corresponding label
        label_name = os.path.splitext(fname)[0] + '.txt'
        shutil.copy2(
            os.path.join(src_dir, "annotations", label_name),
            os.path.join(dst_dir, "annotations", label_name)
        )

In [6]:
# Load the pre-trained YOLO model
data_dir = r'C:/Users/marangi/Desktop/projects/RDD2022/data/United_States'
train_dir = r'C:/Users/marangi/Desktop/projects/RDD2022/data/United_States/train'
val_dir = r'C:/Users/marangi/Desktop/projects/RDD2022/data/United_States/val'
for dir_path in [train_dir, val_dir]:
    os.makedirs(dir_path + "/images", exist_ok=True)
    os.makedirs(dir_path + "/annotations", exist_ok=True)

image_files = [f for f in os.listdir(os.path.join(data_dir, "images")) if f.endswith('.jpg')]
train_files, val_files = train_test_split(image_files, test_size=0.1, random_state=42)
copy_files(train_files, data_dir, train_dir)
copy_files(val_files, data_dir, val_dir)

In [7]:
def train_one_epoch(model, dataloader, optimizer, device):
    model.train()
    total_loss = 0
    
    for batch_idx, (images, targets) in enumerate(tqdm(dataloader)):
        images = images.to(device)
        targets = targets.to(device)
        
        # Forward pass
        loss = model(images, targets)
        
        # Backward pass
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        total_loss += loss.item()
    
    return total_loss / len(dataloader)

def validate(model, dataloader, device):
    model.eval()
    total_loss = 0
    
    with torch.no_grad():
        for images, targets in tqdm(dataloader):
            images = images.to(device)
            targets = targets.to(device)
            
            #forward pass + loss computation
            loss = model(images, targets)
            total_loss += loss.item()
    
    return total_loss / len(dataloader)

In [5]:
# Training setup
train_dataset = YOLO11dataset(
    img_dir=r'C:/Users/marangi/Desktop/projects/RDD2022/data/United_States/train/images', 
    label_dir=r'C:/Users/marangi/Desktop/projects/RDD2022/data/United_States/train/labels',
    img_size=640
)

val_dataset = YOLO11dataset(
    img_dir=r'C:/Users/marangi/Desktop/projects/RDD2022/data/United_States/val/images', 
    label_dir=r'C:/Users/marangi/Desktop/projects/RDD2022/data/United_States/val/labels',
    img_size=640
)

train_loader = DataLoader(train_dataset, batch_size=16, shuffle=True, num_workers=4, pin_memory=True)
val_loader = DataLoader(val_dataset, batch_size=16, shuffle=False, num_workers=4, pin_memory=True)
device = 'cpu' #(ToDo: GPU)
model = YOLO("yolov8n.pt")
num_epochs = 100
hyperparams = {
    'lr0': 0.001,  # initial learning rate for Adam
    'weight_decay': 0.0005,
    'box': 7.5,    # box loss gain
    'cls': 0.5,    # cls loss gain
    'dfl': 1.5,    # dfl loss gain
}
optimizer = torch.optim.Adam(model.parameters(), 
                           lr=hyperparams['lr0'], 
                           weight_decay=hyperparams['weight_decay'])

100%|██████████| 6.25M/6.25M [00:00<00:00, 10.4MB/s]


In [8]:
#Training loop
for epoch in range(num_epochs):
    print(f"Epoch {epoch+1}/{num_epochs}")
    
    # Train
    train_loss = train_one_epoch(model, train_loader, optimizer, device)
    
    # Validate
    val_loss = validate(model, val_loader, device)
    
    print(f"Train Loss: {train_loss:.4f}, Val Loss: {val_loss:.4f}")
    
    # Save checkpoint
    if (epoch + 1) % 10 == 0:
        torch.save({
            'epoch': epoch,
            'model_state_dict': model.state_dict(),
            'optimizer_state_dict': optimizer.state_dict(),
            'train_loss': train_loss,
            'val_loss': val_loss,
        }, f'checkpoint_epoch_{epoch+1}.pt')

# Save final model
torch.save(model.state_dict(), 'final_model.pt')

Epoch 1/100
New https://pypi.org/project/ultralytics/8.3.27 available  Update with 'pip install -U ultralytics'
Ultralytics 8.3.25  Python-3.11.9 torch-2.5.0+cpu CPU (AMD Ryzen 5 4500 6-Core Processor)
engine\trainer: task=detect, mode=train, model=yolov8n.pt, data=coco.yaml, epochs=100, time=None, patience=100, batch=16, imgsz=640, save=True, save_period=-1, cache=False, device=None, workers=8, project=None, name=train8, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, embed=None, show=False, save_fra

100%|██████████| 169M/169M [00:15<00:00, 11.4MB/s] 


KeyboardInterrupt: 

In [11]:
model = YOLO("yolov8n.pt")
results = model.train(
    data = r"C:\Users\marangi\Desktop\projects\RDD2022\data\dataset.yaml",
    epochs = 2,
    imgsz = 640,
    batch = 16,
    device = "cpu"
)

New https://pypi.org/project/ultralytics/8.3.27 available  Update with 'pip install -U ultralytics'
Ultralytics 8.3.25  Python-3.11.9 torch-2.5.0+cpu CPU (AMD Ryzen 5 4500 6-Core Processor)
engine\trainer: task=detect, mode=train, model=yolov8n.pt, data=C:\Users\marangi\Desktop\projects\RDD2022\data\dataset.yaml, epochs=2, time=None, patience=100, batch=16, imgsz=640, save=True, save_period=-1, cache=False, device=cpu, workers=8, project=None, name=train11, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=Fal

train: Scanning C:\Users\marangi\Desktop\projects\RDD2022\data\United_States\train\labels.cache... 3891 images, 0 backgrounds, 0 corrupt: 100%|██████████| 3891/3891 [00:00<?, ?it/s]
val: Scanning C:\Users\marangi\Desktop\projects\RDD2022\data\United_States\val\labels.cache... 433 images, 0 backgrounds, 0 corrupt: 100%|██████████| 433/433 [00:00<?, ?it/s]

Plotting labels to c:\Users\marangi\Desktop\projects\RDD2022\runs\detect\train11\labels.jpg... 


optimizer: 'optimizer=auto' found, ignoring 'lr0=0.01' and 'momentum=0.937' and determining best 'optimizer', 'lr0' and 'momentum' automatically... 
optimizer: AdamW(lr=0.00125, momentum=0.9) with parameter groups 57 weight(decay=0.0), 64 weight(decay=0.0005), 63 bias(decay=0.0)
Image sizes 640 train, 640 val
Using 0 dataloader workers
Logging results to c:\Users\marangi\Desktop\projects\RDD2022\runs\detect\train11
Starting training for 2 epochs...

      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size


        1/2         0G       2.15      3.278      1.874         15        640: 100%|██████████| 244/244 [14:46<00:00,  3.63s/it]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 14/14 [00:39<00:00,  2.84s/it]

...

        2/2         0G      1.886      2.427      1.635          9        640: 100%|██████████| 244/244 [15:12<00:00,  3.74s/it]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 14/14 [00:37<00:00,  2.69s/it]

                   all        433       1019      0.526      0.298      0.249      0.108



2 epochs completed in 0.521 hours.
Optimizer stripped from c:\Users\marangi\Desktop\projects\RDD2022\runs\detect\train11\weights\last.pt, 6.2MB
Optimizer stripped from c:\Users\marangi\Desktop\projects\RDD2022\runs\detect\train11\weights\best.pt, 6.2MB

Validating c:\Users\marangi\Desktop\projects\RDD2022\runs\detect\train11\weights\best.pt...
Ultralytics 8.3.25  Python-3.11.9 torch-2.5.0+cpu CPU (AMD Ryzen 5 4500 6-Core Processor)
Model summary (fused): 168 layers, 3,006,428 parameters, 0 gradients, 8.1 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 14/14 [00:32<00:00,  2.32s/it]


                   all        433       1019      0.529      0.297      0.249      0.108
    Longitudinal_Crack        362        636      0.444      0.568      0.481      0.225
      Transverse_Crack        205        292      0.391       0.25      0.283      0.107
       Alligator_Crack         71         83       0.28       0.37      0.232     0.0999
               Pothole          8          8          1          0          0          0
Speed: 1.4ms preprocess, 61.4ms inference, 0.0ms loss, 3.4ms postprocess per image
Results saved to c:\Users\marangi\Desktop\projects\RDD2022\runs\detect\train11
